# Advanced Language Features

This notebook introduces advanced language features in RAW, which are useful for structuring more complex RQL programs.

In [5]:
%load_ext raw_magic

## Declarations

Declarations are used to organize RQL expressions.

Given the query:

In [6]:
%%rql

SELECT *
FROM READ("https://raw-tutorial.s3.amazonaws.com/airports.csv") a,
     READ("https://raw-tutorial.s3.amazonaws.com/trips.json") t
WHERE t.origin=a.IATA_FAA

... we can instead structure it using declarations as follows:

In [8]:
%%rql

airports := READ("https://raw-tutorial.s3.amazonaws.com/airports.csv");
    
trips := READ("https://raw-tutorial.s3.amazonaws.com/trips.json");
    
SELECT *
FROM airports a,
     trips t
WHERE t.origin=a.IATA_FAA

The queries are equivalent in terms of execution.

Each declaration can be bound to any RAW expression/query.

In [12]:
%%rql

airports := SELECT City, Name, IATA_FAA FROM READ("https://raw-tutorial.s3.amazonaws.com/airports.csv");
    
trips := READ("https://raw-tutorial.s3.amazonaws.com/trips.json");
    
SELECT *
FROM airports a,
     trips t
WHERE t.origin=a.IATA_FAA

## Functions

Functions are similar to declarations, but take arguments as input:

In [15]:
%%rql

airports_by_city(city: string) := SELECT * FROM READ("https://raw-tutorial.s3.amazonaws.com/airports.csv") WHERE City = city;
    
SELECT * FROM airports_by_city("Lisbon")

AirportID,Name,City,Country,IATA_FAA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ
1638,Lisboa,Lisbon,Portugal,LIS,LPPT,38.781311,-9.135919,374,0.0,E,Europe/Lisbon
7752,Lisbon Cruise Terminal,Lisbon,Portugal,,N,38.712606,-9.122483,0,0.0,E,Europe/Lisbon


Functions can have default arguments and can be overridden.

In [21]:
%%rql

airports_by_city(city: string, n: int := 100) := 
    SELECT * FROM READ("https://raw-tutorial.s3.amazonaws.com/airports.csv") WHERE City = city LIMIT n;
    
SELECT * FROM airports_by_city("Lisbon")

AirportID,Name,City,Country,IATA_FAA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ
1638,Lisboa,Lisbon,Portugal,LIS,LPPT,38.781311,-9.135919,374,0.0,E,Europe/Lisbon
7752,Lisbon Cruise Terminal,Lisbon,Portugal,,N,38.712606,-9.122483,0,0.0,E,Europe/Lisbon


In [22]:
%%rql

airports_by_city(city: string, n: int := 100) := 
    SELECT * FROM READ("https://raw-tutorial.s3.amazonaws.com/airports.csv") WHERE City = city LIMIT n;
    
SELECT * FROM airports_by_city("Lisbon", 1)

AirportID,Name,City,Country,IATA_FAA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ
1638,Lisboa,Lisbon,Portugal,LIS,LPPT,38.781311,-9.135919,374,0.0,E,Europe/Lisbon


## Anonymous functions

Anonymous functions, or functions without names, can also be defined.

These use the syntax:
```
\(<list of arguments>) -> <body>
```

For instance:

In [24]:
%%rql

f := \(a: int) -> a + 1;
        
f(1)

int
2


Of course defining an anonymous function then binding it to the name `f` isn't very useful.

Instead, we could have done:

In [25]:
%%rql

f(a: int) := a + 1;
    
f(1)

int
2


However, anonymous functions have a useful feature: they can be the final expression on a virtual view, which effectively makes this into a parameterized view:

In [28]:
%%view view1

\(city: String) -> SELECT * FROM READ("https://raw-tutorial.s3.amazonaws.com/airports.csv") WHERE City = city

We can ask the type of this view:

In [29]:
%query_validate view1

function(
    string,
    collection(
        record(
            AirportID: int,
            Name: string,
            City: string,
            Country: string,
            IATA_FAA: string,
            ICAO: string,
            Latitude: double,
            Longitude: double,
            Altitude: int,
            Timezone: double,
            DST: string,
            TZ: string)))


The view types as a function. So to use it, we can "call it":

In [31]:
%%rql

SELECT * FROM view1("Lisbon")

AirportID,Name,City,Country,IATA_FAA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ
1638,Lisboa,Lisbon,Portugal,LIS,LPPT,38.781311,-9.135919,374,0.0,E,Europe/Lisbon
7752,Lisbon Cruise Terminal,Lisbon,Portugal,,N,38.712606,-9.122483,0,0.0,E,Europe/Lisbon


**Next:** [Packages](Packages.ipynb)